In [1]:
import signac
import numpy as np
import matplotlib.pyplot as plt
import freud
import gsd
import hoomd
import gsd.pygsd
import gsd.hoomd

In [2]:
project = signac.get_project("workspace")
for job in project:
    print(job)

fe36ee4052a01cae041ef5988477de10


In [3]:
state_dict = {"size":"medium", "process":"quench"}

job_list = project.find_jobs(state_dict)

In [4]:
for job in job_list:
    rdf_path = job.fn("trajectory.gsd")

In [5]:
def atom_type_pos(frame, atom_types): 
    positions = []
    for idx, type_id in enumerate(frame.particles.typeid):
        if frame.particles.types[type_id] in atom_types:
            positions.append(frame.particles.position[idx])
    return positions

In [10]:
def create_rdf(rdf_path,
               atom_types='all',
               r_max = None,
               r_min = 0.1,
               nbins = 50,
               start = 100):
    
    
    f = gsd.pygsd.GSDFile(open(rdf_path, "rb"))
    trajectory = gsd.hoomd.HOOMDTrajectory(f) 
    if r_max is None:
        r_max = max(trajectory[-1].configuration.box[:3]) * 0.45 
    freud_rdf = freud.density.RDF(bins=nbins, r_max=r_max, r_min=r_min)
    for frame in trajectory[start:]:
        if atom_types == 'all':
            freud_rdf.compute(system=frame, reset=False)
        else:
            query_points = atom_type_pos(frame, atom_types)
            box = frame.configuration.box
            freud_rdf.compute(system=(box, query_points), reset = False)
        
 
    x = freud_rdf.bin_centers
    y = freud_rdf.rdf
    filename = '{}-trajectory.txt'.format(atom_types[0])
    np.savetxt(job.fn(filename), np.transpose([x,y]), delimiter=',', header= "bin_centers, rdf")
    f.close()
    return freud_rdf, filename


In [11]:
for job in job_list:
    rdf_path = job.fn("trajectory.gsd")
    print (rdf_path)
    create_rdf(rdf_path, atom_types=['f'], start = 95)
    txt_path = job.fn(filename)
    print(txt_path)

/Users/emilyepstein/Projects/ptb7-itic/workspace/fe36ee4052a01cae041ef5988477de10/trajectory.gsd


NameError: name 'filename' is not defined

In [ ]:
for job in job_list:
    txt_path = job.fn(filename)
                      
print(txt_path)

In [ ]:
def rdf_plot(job_list): 

    line = np.genfromtxt(txt_path, names = True, delimiter = ",") 

    x = line["bin_centers"]
    y = line["rdf"]

    plt.plot(x, y)

    plt.xlabel("r")
    plt.ylabel("g(r)")
    #plt.legend(loc="lower right")
    #plt.title("RDF of %s density %smer of PTB7" % (density, polymer_length)) plt.ylim(0, 1.6)

    ymax = max(y)
    n_max = y.argmax()
    plt.plot(x[n_max],y[n_max],'o')
    n_min = y.argmin() 
    plt.plot(x[n_min],y[n_min],'x')

    #something is funky here
    plt.annotate("local max {}".format(n_max), xy=(x[n_max],y[n_max]), xytext=(x[n_max]+15,y[n_max]+15), textcoords='offset points', arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=-0.2"))


    #print("max of", temp,"kT is:") 
    #print((x[n_max],y[n_max]), sep=",")

    #plt.annotate('local min', xy=(x[n_min],y[n_min]), xytext=(x[n_min]+15,y[n_min]+15), textcoords='offset points' , arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=-0.2"))

    #print("min of", temp,"kT is:") 
    #print((x[n_min],y[n_min]), sep=",")
    plt.show()

In [ ]:
rdf_plot(job_list)